<a href="https://colab.research.google.com/github/runnithan03/Deep-Learning-and-Artificial-Intelligence/blob/master/Task%20A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving x_test.csv to x_test.csv
Saving xy_train.csv to xy_train.csv


In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R

if (!require("keras")) install.packages("keras")
if (!require("caret")) install.packages("caret")
library(keras)
library(caret)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: keras
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’, ‘config’, ‘tfautograph’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’

trying URL 'https://cran.rstudio.com/src/contrib/RcppTOML_0.2.3.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/here_1.0.1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/png_0.1-8.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/config_0.3.2.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/tfautograph_0.3.2.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/reticulate_1.42.0.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/tensorflow_2.16.0.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/tfruns_1.5.3.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/zeallot_0.1.0.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/keras_2.15.0.tar.gz'

The downloaded source packages are in


In [ ]:
%%R
raw <- read.csv("x_test.csv")
dim(raw)

[1] 9999  785


In [ ]:
%%R

# Load and preprocess data
xy_train <- read.csv("xy_train.csv", header = FALSE)
x_test <- read.csv("x_test.csv", header = FALSE)

y_train <- as.factor(xy_train[, 1])
X_train_full <- as.matrix(xy_train[, -1])
X_test <- as.matrix(x_test[, -1])

# Normalize
X_train_full <- apply(X_train_full, 2, as.numeric) / 255
X_test <- apply(X_test, 2, as.numeric) / 255

# Labels: A-F → 0-5
y_train <- factor(y_train, levels = c("A", "B", "C", "D", "E", "F"))
y_train_numerical <- as.integer(y_train) - 1


In [ ]:
%%R

dim(x_test)

[1] 10000   785


In [ ]:
%%R

# One-hot encoding
num_classes <- 6
y_train_encoded_full <- matrix(0, nrow = length(y_train_numerical), ncol = num_classes)
for (i in 1:length(y_train_numerical)) {
  y_train_encoded_full[i, y_train_numerical[i] + 1] <- 1
}


In [ ]:
%%R

# Resize and replicate grayscale channels for pretrained model
resize_dim <- 96

reshape_rgb <- function(data) {
  n <- nrow(data)
  data <- array_reshape(data, c(n, 28, 28, 1))
  data <- array(rep(data, 3), dim = c(n, 28, 28, 3))  # grayscale → RGB
  data_resized <- array(0, dim = c(n, resize_dim, resize_dim, 3))
  for (i in 1:n) {
    img <- image_array_resize(data[i,,,], height = resize_dim, width = resize_dim)
    data_resized[i,,,] <- img
  }
  return(data_resized)
}


In [ ]:
%%R

# Split and reshape
set.seed(42)
train_index <- createDataPartition(y_train_numerical, p = 0.8, list = FALSE)

X_train <- reshape_rgb(X_train_full[train_index, ])
X_val <- reshape_rgb(X_train_full[-train_index, ])
X_test <- reshape_rgb(X_test)

y_train_encoded <- y_train_encoded_full[train_index, ]
y_val_encoded <- y_train_encoded_full[-train_index, ]


In [ ]:
%%R

print(dim(X_test))

[1] 10000    96    96     3


In [ ]:
%%R

# Transfer Learning: MobileNetV2
base_model <- application_mobilenet_v2(
  include_top = FALSE,
  weights = "imagenet",
  input_shape = c(resize_dim, resize_dim, 3)
)

freeze_weights(base_model)  # Freeze convolutional base

# Build new model
input <- layer_input(shape = c(resize_dim, resize_dim, 3))
output <- input |>
  base_model() |>
  layer_global_average_pooling_2d() |>
  layer_dropout(rate = 0.5) |>
  layer_dense(units = 128, activation = "relu") |>
  layer_dense(units = num_classes, activation = "softmax")

model <- keras_model(inputs = input, outputs = output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
%%R

# Compile and train
model$compile(
  optimizer = "adam",
  loss = "categorical_crossentropy",
  metrics = list("accuracy")
)

summary(model)

history <- model$fit(
  x = X_train,
  y = y_train_encoded,
  epochs = 20L,
  batch_size = 32L,
  validation_data = list(X_val, y_val_encoded)
)

<Functional name=functional, built=True>
Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 157ms/step - accuracy: 0.3257 - loss: 1.9371
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 399ms/step - accuracy: 0.3347 - loss: 1.9024 - val_accuracy: 0.7475 - val_loss: 0.5137
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - accuracy: 0.7159 - loss: 0.6004
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 240ms/step - accuracy: 0.7162 - loss: 0.5997 - val_accuracy: 0.7576 - val_loss: 0.4797
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 163ms/step - accuracy: 0.7595 - loss: 0.5333
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 219ms/step - accuracy: 0.7589 - loss: 0.5326 - val_accuracy: 0.8485 - val_loss: 0.3576
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.7972 - loss: 0.4138
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.7983 - loss: 0.4124 - val_accuracy: 0.8586 - val_loss: 0.3465
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - accuracy: 0.8286 - loss: 0.3878
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 311ms/step - accuracy: 0.8296 

In [ ]:
%%R

# Confusion Matrix
val_predictions <- model$predict(X_val)
predicted_val_labels <- apply(val_predictions, 1, which.max) - 1
actual_val_labels <- apply(y_val_encoded, 1, which.max) - 1

label_map <- c("A", "B", "C", "D", "E", "F")
predicted_letters <- label_map[predicted_val_labels + 1]
actual_letters <- label_map[actual_val_labels + 1]

confusion <- table(Predicted = predicted_letters, Actual = actual_letters)
print(confusion)
confusionMatrix(confusion)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 723ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 730ms/step
         Actual
Predicted  A  B  C  D  E  F
        A 11  0  0  0  0  0
        B  1 23  0  0  0  0
        C  0  0 15  4  0  0
        D  0  0  1 12  0  0
        E  0  0  0  0 16  5
        F  0  0  0  0  1 10
Confusion Matrix and Statistics

         Actual
Predicted  A  B  C  D  E  F
        A 11  0  0  0  0  0
        B  1 23  0  0  0  0
        C  0  0 15  4  0  0
        D  0  0  1 12  0  0
        E  0  0  0  0 16  5
        F  0  0  0  0  1 10

Overall Statistics
                                          
               Accuracy : 0.8788          
                 95% CI : (0.7978, 0.9358)
    No Information Rate : 0.2323          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.853           
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: A Class

In [ ]:
%%R

# Read raw test data (keep the NAs in the first column)
x_test_raw <- read.csv("x_test.csv", header = FALSE)

# Predict on reshaped X_test
test_predictions <- model$predict(X_test)
predicted_indices <- apply(test_predictions, 1, which.max) - 1
label_map <- c("A", "B", "C", "D", "E", "F")
predicted_letters <- label_map[predicted_indices + 1]

313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 189ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 189ms/step


In [ ]:
%%R

length(predicted_letters)

[1] 10000


In [ ]:
%%R

dim(x_test_raw)
# dim(predicted_letters)

[1] 10000   785


In [ ]:
%%R

# Replace first column with predictions
x_test_raw[, 1] <- predicted_letters

# Save to CSV
write.table(x_test_raw, file = "test_predictions_final.csv", sep = ",",
            row.names = FALSE, col.names = FALSE, quote = FALSE)

# Confirm shape again (optional)
dim(x_test_raw)  # [1] 10000 785

[1] 10000   785


In [ ]:
from google.colab import files
files.download('test_predictions_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%R

dim(output_with_labels)

test <- read.csv("test_predictions.csv", header = FALSE)
dim(test)

Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'output_with_labels' not found


RInterpreterError: Failed to parse and evaluate line '\ndim(output_with_labels)\n\ntest <- read.csv("test_predictions.csv", header = FALSE)\ndim(test)\n'.
R error message: "Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : \n  object 'output_with_labels' not found"

Dump

In [ ]:
%%R

# -----------------------------
# Load and preprocess data
# -----------------------------
xy_train <- read.csv("xy_train.csv")
x_test <- read.csv("x_test.csv")

y_train <- as.factor(xy_train[, 1])
X_train_full <- as.matrix(xy_train[, -1])
X_test <- as.matrix(x_test[, -1])

# Normalize
X_train_full <- apply(X_train_full, 2, as.numeric) / 255
X_test <- apply(X_test, 2, as.numeric) / 255

# Labels: A-F → 0-5
y_train <- factor(y_train, levels = c("A", "B", "C", "D", "E", "F"))
y_train_numerical <- as.integer(y_train) - 1

# One-hot encoding
num_classes <- 6
y_train_encoded_full <- matrix(0, nrow = length(y_train_numerical), ncol = num_classes)
for (i in 1:length(y_train_numerical)) {
  y_train_encoded_full[i, y_train_numerical[i] + 1] <- 1
}

# -----------------------------
# Resize and replicate grayscale channels for pretrained model
# -----------------------------
resize_dim <- 96

reshape_rgb <- function(data) {
  n <- nrow(data)
  data <- array_reshape(data, c(n, 28, 28, 1))
  data <- array(rep(data, 3), dim = c(n, 28, 28, 3))  # grayscale → RGB
  data_resized <- array(0, dim = c(n, resize_dim, resize_dim, 3))
  for (i in 1:n) {
    img <- image_array_resize(data[i,,,], height = resize_dim, width = resize_dim)
    data_resized[i,,,] <- img
  }
  return(data_resized)
}

# Split
set.seed(42)
train_index <- createDataPartition(y_train_numerical, p = 0.8, list = FALSE)

X_train <- reshape_rgb(X_train_full[train_index, ])
X_val <- reshape_rgb(X_train_full[-train_index, ])
X_test <- reshape_rgb(X_test)

y_train_encoded <- y_train_encoded_full[train_index, ]
y_val_encoded <- y_train_encoded_full[-train_index, ]

# -----------------------------
# Transfer Learning: MobileNetV2
# -----------------------------
base_model <- application_mobilenet_v2(
  include_top = FALSE,
  weights = "imagenet",
  input_shape = c(resize_dim, resize_dim, 3)
)

freeze_weights(base_model)  # Freeze convolutional base

# Build new model
input <- layer_input(shape = c(resize_dim, resize_dim, 3))
output <- input |>
  base_model() |>
  layer_global_average_pooling_2d() |>
  layer_dropout(rate = 0.5) |>
  layer_dense(units = 128, activation = "relu") |>
  layer_dense(units = num_classes, activation = "softmax")

model <- keras_model(inputs = input, outputs = output)

# -----------------------------
# Compile and train
# -----------------------------
model$compile(
  optimizer = "adam",
  loss = "categorical_crossentropy",
  metrics = list("accuracy")
)

summary(model)

history <- model$fit(
  x = X_train,
  y = y_train_encoded,
  epochs = 20L,
  batch_size = 32L,
  validation_data = list(X_val, y_val_encoded)
)

# -----------------------------
# Confusion Matrix
# -----------------------------
val_predictions <- model$predict(X_val)
predicted_val_labels <- apply(val_predictions, 1, which.max) - 1
actual_val_labels <- apply(y_val_encoded, 1, which.max) - 1

label_map <- c("A", "B", "C", "D", "E", "F")
predicted_letters <- label_map[predicted_val_labels + 1]
actual_letters <- label_map[actual_val_labels + 1]

confusion <- table(Predicted = predicted_letters, Actual = actual_letters)
print(confusion)
confusionMatrix(confusion)

In [ ]:
%%R

# Predict
predictions <- model$predict(X_test)
predicted_labels <- apply(predictions, 1, which.max) - 1

# Map numeric labels to class names
label_map_reverse <- c("0" = "A", "1" = "B", "2" = "C", "3" = "D", "4" = "E", "5" = "F")
predicted_labels <- label_map_reverse[as.character(predicted_labels)]

# Combine prediction + features, with prediction as the first column
output <- cbind(Prediction = predicted_labels, X_test)

# Save it
write.csv(output, "predictions_with_features.csv", row.names = FALSE)
cat("✅ Predictions (leftmost column) saved to predictions_with_features.csv\n")

In [ ]:
files.download('predictions_with_features.csv')

Previous Way

In [ ]:
%%R

# -----------------------------
# Predict on test set
# -----------------------------
predictions <- model$predict(X_test)
predicted_labels <- apply(predictions, 1, which.max) - 1
label_map_reverse <- c("0" = "A", "1" = "B", "2" = "C", "3" = "D", "4" = "E", "5" = "F")
predicted_labels <- label_map_reverse[as.character(predicted_labels)]

submission <- data.frame(Label = predicted_labels)
write.csv(submission, "predictions.csv", row.names = FALSE)
cat("✅ Predictions saved to predictions.csv\n")

In [ ]:
files.download('predictions.csv')

In [ ]:
%%R

# -----------------------------
# Predict on test set
# -----------------------------
# Get raw prediction matrix
predictions <- model$predict(X_test)

# Display the prediction matrix (or a subset)
cat("Dimensions of prediction matrix:\n")
dim(predictions)  # Shows [n_samples, 6]
cat("\nFirst 5 rows of prediction matrix (probabilities for classes A-F):\n")
print(head(predictions, 5))  # Print first 5 rows for inspection

# Convert to predicted labels
predicted_labels <- apply(predictions, 1, which.max) - 1
label_map_reverse <- c("0" = "A", "1" = "B", "2" = "C", "3" = "D", "4" = "E", "5" = "F")
predicted_labels <- label_map_reverse[as.character(predicted_labels)]

# Create submission dataframe
submission <- data.frame(Label = predicted_labels)
print(head(submission))

In [ ]:
%%R

# Predict
predictions <- model$predict(X_test)
predicted_labels <- apply(predictions, 1, which.max) - 1

# Map numeric labels to class names
label_map_reverse <- c("0" = "A", "1" = "B", "2" = "C", "3" = "D", "4" = "E", "5" = "F")
predicted_labels <- label_map_reverse[as.character(predicted_labels)]

# Combine prediction + features, with prediction as the first column
output <- cbind(Prediction = predicted_labels, X_test)

# Save it
write.csv(output, "predictions_with_features.csv", row.names = FALSE)
cat("✅ Predictions (leftmost column) saved to predictions_with_features.csv\n")